이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.5.1 py4j==0.10.9.7 

Defaulting to user installation because normal site-packages is not writeable


**Spark Session**을 하나 만든다

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark Writing Demo") \
        .master("local[3]") \
        .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.1") \
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/25 16:35:29 WARN Utils: Your hostname, LAPTOP-8CK94L0D resolves to a loopback address: 127.0.1.1; using 172.21.9.191 instead (on interface eth0)
24/06/25 16:35:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/sangwon/devcourse/spark/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sangwon/.ivy2/cache
The jars for the packages stored in: /home/sangwon/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d8672066-d718-4d27-b22a-a5c213fd8f7e;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.5.1 in central
	found org.tukaani#xz;1.9 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.5.1/spark-avro_2.12-3.5.1.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-avro_2.12;3.5.1!spark-avro_2.12.jar (441ms)
downloading https://repo1.maven.org/maven2/org/tukaani/xz/1.9/xz-1.9.jar ...
	[SUCCESSFUL ] org.tukaani#xz;1.9!xz.jar (307ms)
:: resolution report :: resolve 3597ms :: artifacts dl 757ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.5.1 from central in [default]
	org.tukaani#xz;1.9 from central in [default]
	---------------------------------------------------------------------
	|                  |       

In [2]:
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2024-06-25 16:26:01--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 52.92.209.186, 52.92.197.58, 52.218.230.41, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|52.92.209.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv’

appl_stock.csv      100%[===================>] 139.78K   285KB/s    in 0.5s    

2024-06-25 16:26:03 (285 KB/s) - ‘appl_stock.csv’ saved [143130/143130]



In [2]:
df = spark.read \
    .format("csv") \
    .load("appl_stock.csv")

In [3]:
print("Num Partitions before: " + str(df.rdd.getNumPartitions()))
df.groupBy(spark_partition_id()).count().show()

Num Partitions before: 1


+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0| 1763|
+--------------------+-----+



In [4]:
df2 = df.repartition(4)
print("Num Partitions after: " + str(df2.rdd.getNumPartitions()))
df2.groupBy(spark_partition_id()).count().show()

Num Partitions after: 4
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  440|
|                   1|  441|
|                   2|  441|
|                   3|  441|
+--------------------+-----+



In [5]:
df3 = df2.coalesce(2)
print("Num Partitions after: " + str(df3.rdd.getNumPartitions()))
df3.groupBy(spark_partition_id()).count().show()

Num Partitions after: 2
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  881|
|                   1|  882|
+--------------------+-----+



In [6]:
df.write \
    .format("avro") \
    .mode("overwrite") \
    .option("path", "dataOutput/avro/") \
    .save()

In [7]:
df2.write \
    .format("parquet") \
    .mode("overwrite") \
    .option("path", "dataOutput/parquet/") \
    .save()

In [8]:
df3.write \
    .format("json") \
    .mode("overwrite") \
    .option("path", "dataOutput/json/") \
    .save()

In [9]:
!ls -tl dataOutput/

total 12
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 16:36 json
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 16:36 parquet
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 16:36 avro


In [10]:
!ls -tl dataOutput/parquet/

total 112
-rw-r--r-- 1 sangwon sangwon     0 Jun 25 16:36 _SUCCESS
-rw-r--r-- 1 sangwon sangwon 28024 Jun 25 16:36 part-00003-1678275a-e217-445c-a34f-f7762bc364b5-c000.snappy.parquet
-rw-r--r-- 1 sangwon sangwon 27833 Jun 25 16:36 part-00000-1678275a-e217-445c-a34f-f7762bc364b5-c000.snappy.parquet
-rw-r--r-- 1 sangwon sangwon 27787 Jun 25 16:36 part-00001-1678275a-e217-445c-a34f-f7762bc364b5-c000.snappy.parquet
-rw-r--r-- 1 sangwon sangwon 28069 Jun 25 16:36 part-00002-1678275a-e217-445c-a34f-f7762bc364b5-c000.snappy.parquet


In [13]:
!ls -tl dataOutput/avro/

total 88
-rw-r--r-- 1 root root     0 Jan 26 23:31 _SUCCESS
-rw-r--r-- 1 root root 89029 Jan 26 23:31 part-00000-102346ea-bf2b-4535-98bc-72f26c556ee9-c000.avro


In [14]:
!ls -tl dataOutput/json/

total 240
-rw-r--r-- 1 root root      0 Jan 26 23:31 _SUCCESS
-rw-r--r-- 1 root root 122570 Jan 26 23:31 part-00000-7b59cfdd-9394-4373-8445-5014c08f27cb-c000.json
-rw-r--r-- 1 root root 122814 Jan 26 23:31 part-00001-7b59cfdd-9394-4373-8445-5014c08f27cb-c000.json
